## Databricks Data Processing Flow

![Xnip2025-10-05_13-18-33.jpg](./Xnip2025-10-05_13-18-33.jpg "Xnip2025-10-05_13-18-33.jpg")

### What happens on each cluster:

* Unity Catalog acts as central metadata store but not as a Enterprise Multi-version concurrency control (MVCC).
* Driver node **must read Delta table metadata** from storage (S3/ADLS files).
* Each cluster builds its own catalog cache from scratch.
* Query optimizer works with local statistics only.
* No sharing of optimization work between clusters.

###Databricks Delta Lake

* Uses transaction log files (_delta_log/)
* **Optimistic** concurrency control
* Can have write conflicts that users must handle
* Requires understanding of Delta Lake internals

![Xnip2025-10-05_14-28-58.jpg](./Xnip2025-10-05_14-28-58.jpg "Xnip2025-10-05_14-28-58.jpg")

### Databricks: You must:

* Choose isolation level based on consistency needs
* Decide whether to enable [deletion vectors](https://docs.databricks.com/aws/en/delta/deletion-vectors)
* Design partition strategy [Liquid Clustering](https://docs.databricks.com/aws/en/delta/clustering)
  * Backward Compatibility issues:
Tables with Liquid Clustering use Delta writer version 7 and reader version 3, automatically enabling deletion vectors, row tracking, and v2 checkpoints Use liquid clustering for tables | Databricks on AWS. This breaks compatibility
* Understand when conflicts can occur
* Implement retry logic for remaining conflicts

### Trade-offs Summary:

| Setting | Benefit | Cost |
|---------|---------|------|
| Serializable | Perfect consistency | Terrible concurrency |
| Row-level concurrency | Fewer conflicts | Can't use partitions, slight read overhead |
| Partitioning | Fewer conflicts, better pruning | Design complexity, can't change easily |

### Databricks makes users choose between:

Performance OR consistency (isolation levels)
Concurrency OR query performance (row-level vs partitioning)
Flexibility OR simplicity (manual optimization)